## 1. Import libraries and dependencies

In [1]:
from src.preprocess.preprocess import Preprocess
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import time
from src.preprocess.pipeline import Pipeline

/home/hubert/Workspace/Python/UM-Project/languages
{'de': '/home/hubert/Workspace/Python/UM-Project/languages/de', 'en': '/home/hubert/Workspace/Python/UM-Project/languages/en', 'fr': '/home/hubert/Workspace/Python/UM-Project/languages/fr'}
{'de': '/home/hubert/Workspace/Python/UM-Project/languages/de/validated.tsv', 'en': '/home/hubert/Workspace/Python/UM-Project/languages/en/validated.tsv', 'fr': '/home/hubert/Workspace/Python/UM-Project/languages/fr/validated.tsv'}
{'de': '/home/hubert/Workspace/Python/UM-Project/languages/de/clip_durations.tsv', 'en': '/home/hubert/Workspace/Python/UM-Project/languages/en/clip_durations.tsv', 'fr': '/home/hubert/Workspace/Python/UM-Project/languages/fr/clip_durations.tsv'}


2024-05-26 11:12:26.642985: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-26 11:12:27.234290: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## 2. Preprocess audio

In [2]:
ORIGIN_SAMPLE_RATE = 48_000
FINAL_SAMPLE_RATE = 16_000
MAX_CLIENT_ID_AMOUNT = 2500
MIN_CLIP_DURATION_MS = 4000
SET_SIZE = 90_000
BATCH_SIZE = 64

In [3]:
Preprocess.initialize(
    batch_size=BATCH_SIZE,
    max_client_id_amount=MAX_CLIENT_ID_AMOUNT,
    min_clip_duration_ms=MIN_CLIP_DURATION_MS,
    set_size=SET_SIZE,
    origin_sample_rate=ORIGIN_SAMPLE_RATE,
    final_sample_rate=FINAL_SAMPLE_RATE,
)

In [4]:
print(f"Original sample rate: {Preprocess.ORIGIN_SAMPLE_RATE}")
print(f"Final sample rate: {Preprocess.SAMPLE_RATE}")
print(f"Max client id amount: {Preprocess.MAX_CLIENT_ID_AMOUNT}")
print(f"Min clip duration in milliseconds: {Preprocess.MIN_CLIP_DURATION_MS}")
print(f"Set size: {Preprocess.SET_SIZE}")
print(f"One language set size: {Preprocess.ONE_LANG_SET_SIZE}")

print("-"*25 + "SETS SIZES" + "-"*25)
print(f"Training set size: {Preprocess.SET_HPARAMS.train_size}")
print(f"Val set size: {Preprocess.SET_HPARAMS.val_size}")
print(f"Test set size: {Preprocess.SET_HPARAMS.test_size}")
print(f"Is test + val + train is equal one lang set size: "
      f"\n{Preprocess.SET_HPARAMS.test_size + Preprocess.SET_HPARAMS.val_size + Preprocess.SET_HPARAMS.train_size} <= {Preprocess.ONE_LANG_SET_SIZE}")

Original sample rate: 48000
Final sample rate: 16000
Max client id amount: 2500
Min clip duration in milliseconds: 4000
Set size: 90000
One language set size: 30000
-------------------------SETS SIZES-------------------------
Training set size: 18000
Val set size: 6000
Test set size: 6000
Is test + val + train is equal one lang set size: 
30000 <= 30000


In [5]:
# start = time.time()
# Preprocess.preprocess_probes()
# end = time.time()
# print(f'Time taken: {end - start}')

In [6]:
from src.hyperparams.set_hparams import SetHyperparameters
SET_SIZE = 18_000
set_hparams = SetHyperparameters(set_size=SET_SIZE)
print(set_hparams.train_size)
print(set_hparams.val_size)
print(set_hparams.test_size)

10800
3600
3600


In [7]:
import random
train = Preprocess.load_set("val")
val = Preprocess.load_set("val")
test = Preprocess.load_set("test")
random.shuffle(train)
random.shuffle(val)
random.shuffle(test)
train = train[:set_hparams.train_size]
val = val[:set_hparams.val_size]
test = test[:set_hparams.test_size]

In [8]:
len(train), len(val), len(test)

(10800, 3600, 3600)

In [9]:
train_dataset = Pipeline.create_pipeline(train, is_train=True)
val_dataset = Pipeline.create_pipeline(val)
test_dataset = Pipeline.create_pipeline(test)

2024-05-26 11:12:33.419861: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-26 11:12:33.457361: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-05-26 11:12:33.580699: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA


In [ ]:
len(train_dataset), len(val_dataset), len(test_dataset)

# counter = 0
# start = time.time()
# for batch, label in train_dataset:
#     print(batch.shape)
# 
# end = time.time()
# print(f"Train it time: {end - start}")
# print(counter)


In [ ]:
# counter = 0
# start = time.time()
# for batch, label in val_dataset:
#     print(batch.shape)
#     for sample in batch:
#         print(sample)
#         counter += 1
# end = time.time()
# print(f"Val it time: {end - start}")
# print(counter)

In [ ]:
# counter = 0
# start = time.time()
# for batch, label in test_dataset:
#     print(batch.shape)
#     for sample in batch:
#         print(sample)
#         counter += 1
# end = time.time()
# print(f"Test it time: {end - start}")
# print(counter)

In [ ]:
test_dataset.element_spec

In [ ]:
val_dataset.element_spec

In [ ]:
train_dataset.element_spec

## 4. Create model

In [ ]:
# input_shape = test_dataset.as_numpy_iterator().next()[0].shape[1:]
# input_shape

In [ ]:
model = keras.Sequential([
    layers.Input(shape=(497, 257, 1)),
    layers.Conv2D(16, (7, 5), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 1)),
    layers.Conv2D(32, (5, 5), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(3, activation='softmax')
])
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=20, validation_data=val_dataset, batch_size=BATCH_SIZE)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)
print(f'\nTest accuracy: {test_acc}')

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()
